In [1]:
# imports

import pandas as pd
import json
import numpy as np
import random
import re
import os

In [2]:
def clean_text(text):

    # Remove leading and trailing whitespaces and newlines
    text = text.strip()
    
    # Replace multiple consecutive spaces or newlines with a single space or newline
    text = re.sub(r' {2,}', ' ', text)
    text = re.sub(r'\n{2,}', '\n', text)
    
    # Remove literal string '\n'
    text = text.replace(r'\\n', '')
    return text

In [3]:

data_list = []
line_list = []
discard = []
linecount = 0
sources = ['arxiv_', 'peerread_', 'reddit_', 'wikihow_', 'wikipedia_']
llm_list = ['bloomz', 'davinci', 'cohere', 'chatGPT', 'dolly']
keys_human = ['abstract', 'human_text', 'human_reviews']
keys_machine = ['machine_abstract', 'machine_text', 'davinci_reviews', 'chatgpt_reviews', 'bloom_reviews', 'machine_answer']
keys_bloomz = ['bloomz', 'bigscience/bloomz']

In [4]:

human_text_added = False
nanvals = ['NaN', 'nan', None, np.nan, '']

for source in sources:
    for llm in llm_list:
        raw_json = []
        filename = os.path.relpath(f"datasets/M4/{source}{llm}.jsonl")
        with open(filename, 'r', encoding='utf-8') as f:
            try:
                file = f.readlines()
                for line in file:
                    data = json.loads(line)
                    raw_json.append(data)
                if human_text_added == False:
                    for line in raw_json:
                        for key, value in line.items():
                            if value in keys_bloomz:
                                data_list.append({'text' : clean_text(line['text']), 'label' : 'bloomz'})
                            if key in keys_human:
                                if value not in nanvals:
                                    data_list.append({'text': clean_text(value), 'label': 'human'})
                            elif key in keys_machine:
                                    data_list.append({'text': clean_text(value), 'label' : line['model']}) 
                    human_text_added == True
                else:
                    for line in raw_json:
                        for key, value in line.items():
                            if key in keys_machine:
                                if value not in nanvals:
                                    data_list.append({'text': clean_text(value), 'label' : line['model']})              
            except:
                continue
    human_text_added = False


for index, row in df2.iterrows():
    

In [5]:
print(len(data_list))

76065


In [6]:
keylist = []
for line in data_list:
    if line['label'] not in keylist:
        keylist.append(line['label'])
print(keylist)

['human', 'text-davinci-003', 'command-xlarge-nightly', 'gpt-3.5-turbo', 'cohere', 'cohere-xlarge-nightly', 'dolly-v2-12b']


In [7]:
model_dict = {
    'bigscience/bloomz' : 'bloomz',
    'davinci' : 'text-davinci-003',
    'command-xlarge-nightly' : 'cohere', 
    'dolly-v2-12b' : 'dolly-v2', 
    'chatgpt' : 'gpt-3.5-turbo', 
    'cohere-xlarge-nightly' : 'cohere'
}

for line in data_list:
    if line['label'] in model_dict.keys():
        line['label'] = model_dict[line['label']]
            
keylist = []
for line in data_list:
    if line['label'] not in keylist:
        keylist.append(line['label'])
print(keylist)

['human', 'text-davinci-003', 'cohere', 'gpt-3.5-turbo', 'dolly-v2']


In [9]:
print(len(data_list))

101820


In [10]:
df = pd.DataFrame(data_list).astype(pd.StringDtype())
df.shape

(101820, 2)

In [11]:
df.head

<bound method NDFrame.head of                                                      text             label
0       A rather non-standard quantum representation o...             human
1       This paper studies the connection between ordi...  text-davinci-003
2       We discuss the results from the combined IRAC ...             human
3       This work presents an analysis of the c2d Spit...  text-davinci-003
4       Results from spectroscopic observations of the...             human
...                                                   ...               ...
101815  It requires special equipment . It takes up sp...            bloomz
101816  Write a detailed response for the question. Us...            bloomz
101817  Write an abstract for a Wikipedia article with...            bloomz
101818  Write a WikiHow article content given a title ...            bloomz
101819  It doesn't, it sends them to another device th...            bloomz

[101820 rows x 2 columns]>

In [12]:
df = df.sample(frac=1).reset_index(drop=True)
df.drop_duplicates()
df = df[df['text'] == df['text']]
df.shape

(101820, 2)

In [13]:
df.to_csv('dataset_multi.csv', index=False, encoding='utf-8')

In [16]:
# Create the binary classification dataset
# Merge In-The-Wild mixed datasets

filenames = [os.path.relpath("datasets/human-machine-mix/ArticleForge.jsonl"),
             os.path.relpath("datasets/human-machine-mix/Kafkai.jsonl"),
             os.path.relpath("datasets/human-machine-mix/RedditBot.jsonl"),
             os.path.relpath("datasets/human-machine-mix/AI-Writer.jsonl")]

data_list = []
for filename in filenames:
    with open(filename, 'r', encoding='utf-8') as file:
        for line in file:
            # Parse the JSON data
            data = json.loads(line)
            
            # Extract the 'text'
            text = data["text"]
            label = data['label']

            
            # Append the extracted data to the results list
            data_list.append({'text': text, 'label': label})

df = pd.DataFrame(data_list)
df = df.sample(frac=1).reset_index(drop=True)
df = df[df['text'] == df['text']]
df.to_csv('dataset_binary.csv', index=False)